### Redis server to support dashapps

## IF YOU WANT TO SEE WARNINGS, COMMENT THIS OUT

In [1]:
import warnings
warnings.filterwarnings("ignore")

### Imports to support redis udates

In [2]:
import sys
import os

if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))

import pandas as pd

import pyarrow as pa
import redis

import time
import schedule_it



### Imports relating to application

In [3]:
import pandas as pd
import numpy as np

In [4]:
redis_port = 6379
redis_db = redis.Redis(host = 'localhost',port=6379,db=0)

In [5]:
def update_redis_df(key,df):
    context = pa.default_serialization_context()
    redis_db.set(key, context.serialize(df).to_buffer().to_pybytes())


In [6]:
def schedule_updates(hour,update_callback):
    logger = schedule_it.init_root_logger("logfile.log", "INFO")
    while True:
        logger.info(f"scheduling update for hour {hour}")
        sch = schedule_it.ScheduleNext('hour', hour,logger = logger)
        sch.wait()
        logger.info(f"updating history")
        update_db()
        logger.info(f"sleeping for an hour before next scheduling")
        time.sleep(60*60)
    

In [ ]:
def update_db():
    print(f"updating db")
    df_dummy_tester = pd.DataFrame({'x':range(10),'y':[i*10 for i in range(10)]})
    update_redis_df('df_dummy_tester',df_dummy_tester)
    time.sleep(10)

if __name__=='__main__':
    update_db()
    schedule_updates(8,update_db)

updating db


2020-06-23 15:31:59,878 - root - INFO - scheduling update for hour 8
2020-06-23 15:31:59,958 - root - INFO - Sleeping at time 2020-06-23 15:31:59.958291-04:00 for 16.466944 hours


## END